# Report Generation - Rally Progress

Generate a report for each section of a specified rally and each stage in each section.

The report text is appended to either pre-existing report files, or written to newly created report files.

In [7]:
import notebookimport

In [11]:
#!pip3 install unidecode

In [55]:
if __name__=='__main__':
    rebase='overallleader'
    #rebase='PAD'#'PAD'
    MAXINSPLITDELTA=20 #set xlim on the within split delta

In [56]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")
sp = __import__("Charts - Stage Progress")

In [57]:
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english
from pytracery_logic import pytracery_logic

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        
    grammar = tracery.Grammar(rules)
        
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

In [58]:
#need to get db builder -
#eg http://localhost:8888/notebooks/Documents/code/github%20forks/WRC_sketches/doodles/WRC%20API%202018%20Base.ipynb

In [133]:
import os
import sqlite3
import pandas as pd
import pytablewriter
import six

#dbname='wrc18.db'
#dbname='france18.db'
#conn = sqlite3.connect(dbname)

if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='spain18.db'
    conn = sqlite3.connect(dbname)
    rally='Spain'
    rc='RC1'
    year=2018
    #ss='SS4'

In [12]:
basepath = 'report'
imgdir = 'images'
imgdirfull = '{}/{}'.format(basepath, imgdir)


In [8]:
#rally='France'

In [93]:
'''year=2018
rc='RC2'
ss='SS4'


typ='stage_times_stage' #stage_times_stage stage_times_overall
typ='stage_times_overall'''

"year=2018\nrc='RC2'\nss='SS4'\n\n\ntyp='stage_times_stage' #stage_times_stage stage_times_overall\ntyp='stage_times_overall"

In [134]:
if __name__=='__main__':
    stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')

    #The getEnrichedStageRank query filters down to a class - default is RC1
    stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')

    stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')

This is a simple example of the sort of report string we want to report against.

```python
txt='In stage {stage}, {lead} took the stage, {nthstagewin}, in a time of {time}.\n\nIn second place on stage, {diff} behind, {second} retained overall lead of the rally.'.format(**row)
```

In [95]:
stagerank_stage.iloc[10]

diffFirst                          00:00:15.9000000
diffFirstMs                                   15900
diffPrev                           00:00:00.1000000
diffPrevMs                                      100
elapsedDuration                    00:03:51.2000000
elapsedDurationMs                            231200
entryId                                        3049
position                                         27
source                                      Default
stageId                                         823
stageTimeId                                   45361
status                                    Completed
class                                           RC1
code                                            SS1
distance                                        3.2
name                    Barcelona (asphalt) TV Live
snum                                              1
driver.code                                     LOE
entrant.name           CITROËN  TOTAL ABU DHABI WRT
classrank   

In [13]:
sections = stageresults.groupby('section')

#Generate list from group: stageresults.groupby('section')['code'].apply(list)

#Generate group then iterate through groups
for key in sections.groups.keys():
    print(key, sections.get_group(key)['code'].tolist())


Section 10 ['SS17']
Section 11 ['SS18']
Section 2 ['SS1']
Section 3 ['SS2', 'SS3', 'SS4']
Section 4 ['SS5', 'SS6', 'SS7']
Section 5 ['SS8', 'SS9', 'SS10']
Section 6 ['SS11', 'SS12', 'SS13']
Section 7 ['SS14']
Section 8 ['SS15']
Section 9 ['SS16']


In [102]:
sectionREADME_base = '''\n### {section} Report
'''

## Section Reports

Can these be limited to e.g. just RC1?

TO DO - need something if the section is a single stage - not s split chart, just a spot chart?

In [103]:
cols=['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'elapsedDuration',
       'elapsedDurationMs', 'entryId', 'position', 'source', 'stageId',
       'stageTimeId', 'status', 'class', 'code', 'distance', 'name', 'snum',
       'driver.code', 'entrant.name', 'classrank', 'gainedClassPos',
       'gainedClassLead', 'classPosDiff', 'lostClassLead', 'retainedClassLead',
       'gainedOverallPos', 'gainedOverallLead', 'overallPosDiff',
       'lostOverallLead', 'retainedOverallLead', 'stagewin', 'stagewincount',
       'winsinarow', 'gainedTime']
cols=['drivercode','entrant.name','elapsedDuration','position','classrank' ,'diffFirst', 'diffPrev']

#stagerank_stage[stagerank_stage['code']=='SS9'][cols].sort_values('position')

In [104]:
q='''
    SELECT itc.code, itc.status
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND itc.code LIKE "SS%" 
    '''.format(rally=rally)
stage_status=pd.read_sql(q,conn).set_index('code').to_dict(orient='index')
stage_status

{'SS10': {'status': 'Completed'},
 'SS11': {'status': 'Completed'},
 'SS12': {'status': 'Completed'},
 'SS13': {'status': 'Running'},
 'SS14': {'status': 'ToRun'},
 'SS15': {'status': 'ToRun'},
 'SS16': {'status': 'ToRun'},
 'SS17': {'status': 'ToRun'},
 'SS9': {'status': 'Completed'},
 'SS1': {'status': 'Completed'},
 'SS2': {'status': 'Completed'},
 'SS3': {'status': 'Completed'},
 'SS18': {'status': 'ToRun'},
 'SS4': {'status': 'Interrupted'},
 'SS5': {'status': 'Completed'},
 'SS6': {'status': 'Completed'},
 'SS7': {'status': 'Completed'},
 'SS8': {'status': 'Cancelled'}}

In [135]:
%%capture
if not os.path.exists(imgdirfull):
    os.makedirs(imgdirfull)


#Maybe have a dataframe styling / formatting / output notebook?
def _markdown_table_write(tmp_df, prefix='\n\n'):
    ''' Write out pandas dataframe as a markdown table. '''
    writer = pytablewriter.MarkdownTableWriter()
    writer.stream = six.StringIO()
    
    #Drop drivers without a position
    writer.from_dataframe(tmp_df)
    writer.write_table()
    return prefix+writer.stream.getvalue()


def get_overall_results_table(stagerank_overall,ss):
    ''' Retrieve overall rank data and return it as a markdown formatted table. '''
    
    cols=cols=['drivercode','entrant.name','totalTime','position','classrank' ,'diffFirst', 'diffPrev']
    tmp_df = stagerank_overall[stagerank_overall['code']==ss][cols].sort_values('position')

    tmp_df.columns=['Driver','Team','Elapsed Duration','Position','Class Rank' ,'diffFirst', 'diffPrev']  
    
    #Hack the markdown to get rid of the trailing zeroes in the millisecond representation
    return _markdown_table_write(tmp_df.dropna(subset=['Position'])).replace('000000','')


def get_stage_results_table(stagerank_stage,ss):
    ''' Retrieve stage rank data and return it as a markdown formatted table. '''
    
    cols=['drivercode','entrant.name','elapsedDuration','position','classrank' ,'diffFirst', 'diffPrev']
    tmp_df = stagerank_stage[stagerank_stage['code']==ss][cols].sort_values('position')
    tmp_df.columns=['Driver','Team','Elapsed Duration','Position','Class Rank' ,'diffFirst', 'diffPrev']  
    
    #Hack the markdown to get rid of the trailing zeroes in the millisecond representation
    return _markdown_table_write(tmp_df.dropna(subset=['Position'])).replace('000000','')
    #writer = pytablewriter.MarkdownTableWriter()
    #writer.stream = six.StringIO()
    
    ##Drop drivers without a position
    #writer.from_dataframe(tmp_df.dropna(subset=['Position']))
    #writer.write_table()
    #return '\n\n'+writer.stream.getvalue()

    
def sectionRankingChart(data, title, sectionREADME, key, sectionStages, fn_core, deltalabels=True, elaborate=True):
    ''' Generate section ranking chart, save as image file, and add embedded image link to sectionREADME text. '''
    data = sr.stageOverallEnrichers(data)
    ss = data[data['code'].isin(sectionStages)].copy()
    ss = sr.stageLegEnrichers(ss)
    if ss.empty or ss.dropna(subset=['diffFirst']).empty: return sectionREADME
    fn='{}/{}'.format(imgdirfull, fn_core)
    fig, ax = sr.plotStageProgressionChart( ss, linecolor='lightgrey' , deltalabels=deltalabels, elaborate=elaborate,
                                        progress=True, stageAnnotate=False, filename=fn, title = title); 
    if fig is not None:
        fn='{}/{}'.format(imgdir, fn_core)
        sectionREADME = '''\n\n{s}{key}\n\n![]({img})\n'''.format(s=sectionREADME, key=key,img=fn)
    return sectionREADME


def _sectionStagesReport(sections, stagerank_overall, stagerank_stage, basepath, rebase='overall'):
    ''' Generate stage sections reports. '''
    
    #TO DO - these aren't yet available as rebased reports, if that event makes sense?
    #The table could be rebased
    dn ='_'+rebase if rebase and 'overall' not in rebase else ''
        
    for key in sections.groups.keys():
        #path = '{}/{}'.format(basepath,key)
        #if not os.path.exists(path):
        #    os.makedirs(path)

        out_file_name = '{bp}/{key}_report{dn}.md'.format(bp=basepath, key=key, dn=dn)

        sectionStages = sections.get_group(key)['code'].tolist()
        print('Section stages: {}'.format(sectionStages))
        #Check section stages - if TORUN, then break
        
        sectionREADME = sectionREADME_base.format(section=key)
        
        #Stage rank chart
        fn_core='spchart_stage_{}.png'.format( key)
        title = '{} - Stage Ranking'.format(key)
        sectionREADME = sectionRankingChart(stagerank_stage, title, sectionREADME,
                                            key, sectionStages, fn_core, deltalabels=False, elaborate=False)

        #Overall rally ranking chart
        fn_core='spchart_overall_{}.png'.format( key)
        title = '{} - Overall Rally Ranking Evolution'.format(key)
        sectionREADME = sectionRankingChart(stagerank_overall, title, sectionREADME,
                                            key, sectionStages, fn_core, deltalabels=True, elaborate=True )

        #Display results as table
        # TO DO - Need to do something about stage vs rally overall result?
        for stage in sectionStages:
            if stage_status[stage]['status']=='ToRun': break
            #print(stage)
            stage_result_table = get_stage_results_table(stagerank_stage,stage)
            sectionREADME = '{}\n\n##Stage Result - {}\n\n{}\n\n'.format(sectionREADME, stage, stage_result_table)

            #TO DO - do we need to over the ability to rebase this table?
            overall_result_table = get_overall_results_table(stagerank_overall,stage)
            sectionREADME = '{}\n\n##Overall Result - {}\n\n{}\n\n'.format(sectionREADME, stage, overall_result_table)

            

        with open(out_file_name, 'a') as out_file:
            out_file.write(sectionREADME)

def sectionStagesReport(conn, rally, basepath, rebase='overall'):
    stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')
    stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')
    stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')
    sections = stageresults.groupby('section')

    _sectionStagesReport(sections, stagerank_overall, stagerank_stage, basepath, rebase)

sectionStagesReport(conn, rally, basepath, rebase=rebase)#'overall')

### Stage progression chart

In [136]:
%%capture

#This is a step in the creation of output files used to generate the rally report

# Generate the stage progression chart for the extent of the rally to date
fn='{}/spchart_full.png'.format(imgdirfull)
fig, ax = sr.plotStageProgressionChart( stagerank_overall, linecolor='lightgrey' , deltalabels=True, 
                                        progress=True, stageAnnotate=False, filename=fn);


#what do we need to add to allow stageannotate to be set True?
#fn='{}/spchart_full2.png'.format(imgdirfull)
#fig, ax = sr.plotStageProgressionChart( stagerank_overall, 
#                                        progress=True, stageAnnotate=True, filename=fn);

#Need to find a way to check we don't add the link repeatedly to the file
if 'spchart_full.png' not in open('{}/README.md'.format(basepath)).read()
    README='''\n![]({})'''.format('{}/spchart_full.png'.format(imgdir))
    with open('{}/README.md'.format(basepath), 'a') as out_file:
        out_file.write(README)

In [107]:
!ls report

README.md                Section 10_report_PAD.md Section 7_report.md
SS10_report.md           Section 11_report.md     Section 7_report_PAD.md
SS11_report.md           Section 11_report_PAD.md Section 8_report.md
SS1_report.md            Section 1_report.md      Section 8_report_PAD.md
SS2_report.md            Section 2_report.md      Section 9_report.md
SS3_report.md            Section 2_report_PAD.md  Section 9_report_PAD.md
SS4_report.md            Section 3_report.md      _book
SS5_report.md            Section 3_report_PAD.md  docs
SS6_report.md            Section 4_report.md      entries_RC1.md
SS7_report.md            Section 4_report_PAD.md  entries_RC2.md
SS8_report.md            Section 5_report.md      entries_RC3.md
SS9_report.md            Section 5_report_PAD.md  entries_RC4.md
SUMMARY.md               Section 6_report.md      images
Section 10_report.md     Section 6_report_PAD.md  maps


## Stage Reports

In [108]:
#overallleader = stagerank_overall[stagerank_overall['position']==1].set_index('code')[['drivercode']].to_dict(orient='dict')['drivercode']
#overallleader

In [109]:
if __name__=='__main__':
    sections.groups.keys()

dict_keys(['Section 10', 'Section 11', 'Section 2', 'Section 3', 'Section 4', 'Section 5', 'Section 6', 'Section 7', 'Section 8', 'Section 9'])

In [110]:
if __name__=='__main__':
    #Look for maps
    #Quick sketch to see what maps are available and what stages they refer to
    #The maps are pulled from http://localhost:8888/notebooks/Documents/GitHub/wrcplus/notebooks/Map%20KML%20etc.ipynb
    # TO DO: bring stage map capture creation tools into this workflow
    maps={}
    for i in os.listdir('report/maps'):
        maps[i] = i.split('.')[0].replace('0','').replace('-','-SS').replace('SSS','SS')
    smaps = {}
    for m in maps:
        for i in maps[m].split('-'):
            smaps[i] = m
    smaps

{'SS3': 'SS3-6.png',
 'SS6': 'SS3-6.png',
 'SS15': 'SS15-17.png',
 'SS17': 'SS15-17.png',
 'spain_218': 'spain_2018.geojson',
 'Shakedown': 'Shakedown.png',
 'SS9': 'SS9-12.png',
 'SS12': 'SS9-12.png',
 'SS1': 'SS10-13.png',
 'SS2': 'SS2-5.png',
 'SS5': 'SS2-5.png',
 'SS16': 'SS16-18.png',
 'SS18': 'SS16-18.png',
 'SS4': 'SS4-7.png',
 'SS7': 'SS4-7.png',
 'SS8': 'SS8-11.png',
 'SS11': 'SS8-11.png',
 'SS13': 'SS10-13.png',
 'SS14': 'SS14.png'}

In [23]:
#stagecodes

In [24]:
chartexplainer={}
chartexplainer['stageprogresschart']= \
'''
The stage progress chart shows times rebased relative to the specified driver and highlights:

- the overall gap relative to other drivers at the start of the stage;
- the time gained / lost relative to other drivers on the stage;
- the overall gap relative to other drivers at the end of the stage.
'''

chartexplainer['splitdeltaoverallchart']= \
'''
For the specified driver, show the overall stage time delta at each split relative to the other drivers.
'''

chartexplainer['splitdeltawithinchart']= \
'''
For the specified driver, show the delta for each split section relative to the other drivers.
'''

#chartexplainer['splitdeltaoverchart']= \
#'''
#'''

In [25]:
#stagecodes

SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
SS11
SS12
SS13


In [ ]:
import matplotlib.pyplot as plt
import unidecode

stagecodes = stageresults['code']

def stageReport(stagerank_overall,stagerank_stage, stagecodes, stage_status, rebase='overallleader', forceAll=False  ):
    drivercodes = stagerank_overall['driver.code'].unique()
    links=[]
    overallleader = stagerank_overall[stagerank_overall['position']==1].set_index('code')[['driver.code']].to_dict(orient='dict')['driver.code']
    for ss in stagecodes:
        #Check section stages - if TORUN, then break
        if not forceAll and stage_status[ss]['status']=='ToRun': break

        print(ss)
        
        dn=''
        invert_colours=False
        if rebase=='overallleader':
            dc = overallleader[ss]
        elif rebase in drivercodes:
            dc = rebase
            dn = '_'+dc
            invert_colours=True
        else:
            dc = overallleader[ss]
            
        #Should we avoid SS1?
        #SS1 stage bars are borked - disable for now?
        #if ss=='SS1': continue
        currprevstagerank = sp.getCurrPrevStageRank(conn, rally, rc, ss)
        splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
        # TO DO - SS1 stage bars are borked - disable for now?
        #Need to sort them acc. to driver who ends up first at SS1 end?
        if ss=='SS1': splitdurations=None
        with open('{}/{}_report{}.md'.format(basepath, ss, dn), 'w') as f:
            #ADD STAGE MAP IMAGE TO REPORT
            if ss in smaps:
                f.write('# Stage Map - {rally}, {year} - SS{ss} - Relative Split Times\n\n'.format(year=year,rally=rally,
                                                                                  ss=str(ss).replace('SS','')))
                img='maps/{}'.format(smaps[ss])
                f.write('![]({})\n'.format(img))

            #for dc in currprevstagerank['drivercode'].unique():
            
            splits = ssd.dbGetSplits(conn,rally,ss,rc)
            #If we have split times for this stage, report on them
            if splits is not None and not splits.empty :
                
                #TABLE IMAGE - borked atm if there are no splits
                s2 = ds.getDriverSplitsReport(conn, rally, ss, dc, rc, 'overall',
                                              order='previous', caption='Rally Australia {} - {}'.format(ss, dc))
                                              #order: overall | previous | roadpos
                _img=ds.getTablePNG(s2,basepath=basepath,
                                    path=imgdir,
                                    fnstub='driver_splits_table_{ss}_{dc}'.format(ss=ss,dc=unidecode.unidecode(dc)))
                f.write('\n![]({})\n'.format(_img))

                #print(splits)
                elapseddurations=ssd.getElapsedDurations(splits)
                try:
                    if elapseddurations is not None and not elapseddurations.empty:
                        fig, ax = ssd.plotSplitOverallDelta(elapseddurations,dc,invert_colours=invert_colours,xmin=-180, xmax=180)
                        img='{}/stage_report_split_delta_{}_{}.png'.format(imgdir,str(ss).replace('SS',''), unidecode.unidecode(dc))
                        fig.savefig('{}/{}'.format(basepath,img))   # save the figure to file
                        plt.close(fig)  
                        f.write('# Stage Overall Split Delta Chart - {rally}, {year} - {dc} - SS{ss}\n\n'.format(year=year,rally=rally,
                                                                                              dc=dc,
                                                                                              ss=str(ss).replace('SS','')))
                        f.write(chartexplainer['splitdeltaoverallchart'])
                        f.write('![]({})\n'.format(img))
                except:
                    print("Error with Overall Split Delta Chart {}".format(ss))

                #try:
                splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
                if splitdurations is not None and not splitdurations.empty:
                    fig, ax = ssd.plotSplitSectionDelta(splitdurations,dc,invert_colours=invert_colours, maxdelta=MAXINSPLITDELTA,
                                                        xmin=-180, xmax=180,
                                                        title='Delta within each split - {} - {}'.format(str(ss),unidecode.unidecode(dc)))
                    img='{}/stage_report_individual_split_delta_{}_{}.png'.format(imgdir,str(ss).replace('SS',''), unidecode.unidecode(dc))
                    fig.savefig('{}/{}'.format(basepath,img))   # save the figure to file
                    plt.close(fig)  
                    f.write('# Stage Within Split Delta Chart - {rally}, {year} - {dc} - SS{ss}\n\n'.format(year=year,rally=rally,
                                                                                          dc=dc,
                                                                                          ss=str(ss).replace('SS','')))
                    f.write(chartexplainer['splitdeltawithinchart'])
                    f.write('![]({})\n'.format(img))
                #except:
                #    print("Error with Within Split Delta Chart {}".format(ss))

            fig, ax = sp.rebaseStageProgressBar(currprevstagerank, dc, splitdurations=splitdurations,
                                    neg='lightgreen',pos='pink',invert_colours=invert_colours,xmin=-180, xmax=180,
                         title='Stage Progress Chart - {rally}, {year} - SS{ss}'.format(year=year,
                                                                                        rally=rally,
                                                                                        ss=str(ss).replace('SS','')));
            img='{}/stage_report_{}_{}.png'.format(imgdir,str(ss).replace('SS',''), unidecode.unidecode(dc))
            fig.savefig('{}/{}'.format(basepath,img))   # save the figure to file
            plt.close(fig)  
            f.write('# Stage Progress Chart - {rally}, {year} - {dc} - SS{ss}\n\n'.format(year=year,rally=rally,
                                                                                  dc=dc,
                                                                                  ss=str(ss).replace('SS','')))
            explainer = chartexplainer['stageprogresschart']
            f.write(explainer)
            f.write('![]({})\n'.format(img))

            
            stage_result_table = get_stage_results_table(stagerank_stage,ss)
            f.write(stage_result_table)
            
            #TO DO - there is no sense of the overall standing at the end of the stage
            #Need to add an overall standing / results table on each stage report
            
            
            #Can we do a rich table into gitbook?
            #Or save as an image?
            #Or embed as an iframe?
            #s2 = ds.getDriverSplitsReport(conn, rally, 'SS12', 'LAT', rc, 'overall')
            #f.write(s2)

                
#stageReport(stagerank_overall, stagecodes, stage_status, rebase='overallleader'  )
#stageReport(stagerank_overall, stagecodes, stage_status, rebase='PAD'  )

stageReport(stagerank_overall, stagerank_stage, stagecodes, stage_status, rebase=  rebase)
#stageReport(stagerank_overall, stagerank_stage, ['SS17','SS18'], stage_status)


In [112]:
stagerank_overall['code'].unique()

array(['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9',
       'SS10', 'SS11', 'SS12', 'SS13'], dtype=object)

In [113]:
ssd.dbGetSplits(conn,rally,'SS10',rc='RC1').empty
#ssd.getSplitDurationsFromSplits(conn,rally,'SS9',rc)

False

In [114]:
!pwd

/Users/tonyhirst/Documents/GitHub/WRC_sketches/doodles


In [115]:
#!/Users/ajh59/anaconda3/bin/pip install pytablewriter

In [116]:
#splitdurations.head()

In [117]:
#import pytablewriter

#writer = pytablewriter.MarkdownTableWriter()
#writer.from_dataframe(splitdurations.head())
#writer.write_table()

In [138]:
!(cd report/; rm -rf docs; gitbook build;  mv _book docs)

info: 7 plugins are installed 
info: 6 explicitly listed 
info: loading plugin "highlight"... OK 
info: loading plugin "search"... OK 
info: loading plugin "lunr"... OK 
info: loading plugin "sharing"... OK 
info: loading plugin "fontsettings"... OK 
info: loading plugin "theme-default"... OK 
info: found 29 pages 
info: found 71 asset files 
info: >> generation finished with success in 2.4s ! 


In [119]:
#In reports directory
!(cd report/; rm -rf docs; gitbook build;  mv _book docs)

#gitbook serve

info: 7 plugins are installed 
info: 6 explicitly listed 
info: loading plugin "highlight"... OK 
info: loading plugin "search"... OK 
info: loading plugin "lunr"... OK 
info: loading plugin "sharing"... OK 
info: loading plugin "fontsettings"... OK 
info: loading plugin "theme-default"... OK 
info: found 29 pages 
info: found 68 asset files 
info: >> generation finished with success in 2.5s ! 


In [43]:
!ls report/docs

SS10_report.html         Section 1_report.html    Section 8_report_PAD.md
SS1_report.html          Section 2_report.html    Section 9_report.html
SS2_report.html          Section 2_report_PAD.md  Section 9_report_PAD.md
SS3_report.html          Section 3_report.html    entries_RC1.html
SS4_report.html          Section 3_report_PAD.md  entries_RC2.html
SS5_report.html          Section 4_report.html    entries_RC3.html
SS6_report.html          Section 4_report_PAD.md  entries_RC4.html
SS7_report.html          Section 5_report.html    gitbook
SS8_report.html          Section 5_report_PAD.md  images
SS9_report.html          Section 6_report.html    index.html
Section 10_report.html   Section 6_report_PAD.md  maps
Section 10_report_PAD.md Section 7_report.html    search_index.json
Section 11_report.html   Section 7_report_PAD.md
Section 11_report_PAD.md Section 8_report.html


In [202]:
df = sr.dbGetStageStart(conn, rally, rc, stages=14)
df

,stageId,entryId,driver.code,startDateTime,eventId,name,snum,position,diffFirstMs,diffPrevMs
